<a href="https://colab.research.google.com/github/gjrsas/DAEN690/blob/main/AtlanticFlyawayDatabook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#atlantic flyway databook
from google.colab import files
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#File stored in GitHub LFS
url = 'https://github.com/gjrsas/DAEN690/raw/main/Data/WingData.csv'

df = pd.read_csv(url)
print(df.shape)

<ipython-input-1-8b37bb7681d1>:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


(5900720, 21)


In [8]:
# Filter for Atlantic Flyway from 1999-2021 for the Mallard X Black Duck Hybrid species
atlantic_df = df[
    (df['flyway_name'] == 'Atlantic Flyway') &
    (df['species_name'] == 'Mallard X Black Duck Hybrid') &
    (df['harvest_year'] >= 1999) &
    (df['harvest_year'] <= 2021)
]

# Round the harvest_weight to the nearest hundred for Atlantic Flyway
atlantic_df['harvest_weight'] = np.round(atlantic_df['harvest_weight'], -2)

# Aggregate harvest_weight by year for the Atlantic Flyway
atlantic_totals = atlantic_df.groupby(['harvest_year', 'state'])['harvest_weight'].sum().reset_index()
# Convert harvest_weight to integers
atlantic_totals['harvest_weight'] = atlantic_totals['harvest_weight'].astype(int)
#create pivot tables
atlantic_pivot = atlantic_totals.pivot(index='harvest_year', columns='state', values='harvest_weight').fillna(0)


# Define a function to sum harvest weights by year for a specified flyway
def sum_harvest_by_flyway(flyway_name):
    flyway_df = df[
        (df['flyway_name'] == flyway_name) &
        (df['species_name'] == 'Mallard X Black Duck Hybrid') &
        (df['harvest_year'] >= 1999) &
        (df['harvest_year'] <= 2021)
    ].copy()

    flyway_df['harvest_weight'] = np.round(flyway_df['harvest_weight'], -2)
    yearly_sum = flyway_df.groupby('harvest_year')['harvest_weight'].sum().astype(int)

    return yearly_sum

# Sum harvest weights for each flyway
af_totals = sum_harvest_by_flyway('Atlantic Flyway').astype(int)
mf_totals = sum_harvest_by_flyway('Mississippi Flyway').astype(int)
pf_totals = sum_harvest_by_flyway('Pacific Flyway').astype(int)
cf_totals = sum_harvest_by_flyway('Central Flyway').astype(int)

# Merge the totals with the Atlantic Flyway DataFrame
atlantic_pivot['AF'] = af_totals
atlantic_pivot['MF'] = mf_totals
atlantic_pivot['PF'] = pf_totals
atlantic_pivot['CF'] = cf_totals

# Fill missing values with 0
atlantic_pivot.fillna(0, inplace=True)
atlantic_pivot = atlantic_pivot.astype(int)

# Define the time periods for which you want to calculate averages
time_periods = {
    '1999-2000': (1999, 2000),
    '2001-05': (2001, 2005),
    '2006-10': (2006, 2010),
    '2011-15': (2011, 2015),
    '2016-20': (2016, 2020)
}

# Ensure 'harvest_year' is a column
atlantic_pivot.reset_index(inplace=True)

# Sort the data by year first (numeric only)
atlantic_pivot['harvest_year'] = atlantic_pivot['harvest_year'].astype(str)
numeric_years = atlantic_pivot[atlantic_pivot['harvest_year'].str.isnumeric()].copy()
numeric_years['harvest_year'] = numeric_years['harvest_year'].astype(int)
numeric_years.sort_values(by='harvest_year', inplace=True)

# Calculate averages for each time period and create a DataFrame for them
averages_rows = []
for label, (start_year, end_year) in time_periods.items():
    # Select the data for the time period
    period_data = numeric_years[(numeric_years['harvest_year'] >= start_year) & (numeric_years['harvest_year'] <= end_year)]

    # Calculate the mean for each column
    averages = period_data.mean().round(0).astype(int)  # Round to 0 decimal places
    averages['harvest_year'] = label  # Set the label for the averages row
    averages_rows.append(averages)

# Convert the averages list of series to a DataFrame
averages_df = pd.DataFrame(averages_rows)

# Concatenate the yearly data with the averages, placing averages after 2021
final_df = pd.concat([numeric_years, averages_df], ignore_index=True)

# Save the result to a CSV file
final_df.to_csv('AF_Mallard_X_Black_Duck_Hybrid.csv', index=False)

# Display the DataFrame
print(final_df.tail(15))  # Show the last 15 rows to include all averages and the 2021 data

<ipython-input-8-eefeaea19ec7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atlantic_df['harvest_weight'] = np.round(atlantic_df['harvest_weight'], -2)


state harvest_year  CT   DE   FL   GA   MA    MD    ME   NC   NH   ...   PA   \
13            2012    0  500    0    0    0     0   300  800    0  ...   500   
14            2013    0  600  100    0    0   800   400  300  300  ...  1200   
15            2014    0  100  100  100    0   400     0  400    0  ...   100   
16            2015    0  100    0    0    0   300     0  200    0  ...   600   
17            2016    0  400  200    0    0   100     0    0    0  ...   200   
18            2017    0  100    0    0    0   100     0  500    0  ...   400   
19            2018  100  200    0    0    0   300   100  100    0  ...   200   
20            2019  100  300    0    0    0   200     0  400    0  ...   300   
21            2020    0  300  100    0  100   400   400  400    0  ...   100   
22            2021    0    0    0    0    0   400   200    0    0  ...     0   
23       1999-2000  600  500    0  200  150  1500  1550  400    0  ...  1200   
24         2001-05  460  880   80   40  

# New Section